a) Describir brevemente la cantidad de valores faltantes para cada una de las variables de la base de datos.

In [10]:
import pandas as pd

# Cargar datasets
train = pd.read_csv('/content/sample_data/train.csv')
test = pd.read_csv('/content/sample_data/test.csv')

# Verificar columnas y valores faltantes en el conjunto de datos de entrenamiento
print("Columnas en el conjunto de datos de entrenamiento:", train.columns)
print("Valores faltantes en el conjunto de datos de entrenamiento:")
print(train.isnull().sum())

# Verificar columnas y valores faltantes en el conjunto de datos de prueba
print("\nColumnas en el conjunto de datos de prueba:", test.columns)
print("Valores faltantes en el conjunto de datos de prueba:")
print(test.isnull().sum())




Columnas en el conjunto de datos de entrenamiento: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Valores faltantes en el conjunto de datos de entrenamiento:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Columnas en el conjunto de datos de prueba: Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Valores faltantes en el conjunto de datos de prueba:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


b) Completar aquellas variables que se encuentran faltantes para las bases de train (“Age”, “Fare”, “Cabin”) y test (“Age”, “Cabin”, “Embarked”).

In [6]:
# Imputar valores faltantes en el conjunto de datos de entrenamiento
train['Age'].fillna(train['Age'].median(), inplace=True)
train['Fare'].fillna(train['Fare'].median(), inplace=True)
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)
train.drop('Cabin', axis=1, inplace=True)  # Eliminamos la columna 'Cabin' debido a demasiados valores faltantes

# Imputar valores faltantes en el conjunto de datos de prueba
test['Age'].fillna(test['Age'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)
test.drop('Cabin', axis=1, inplace=True)  # Eliminamos la columna 'Cabin' debido a demasiados valores faltantes

# Verificar que ya no hay valores faltantes
print("Valores faltantes en el conjunto de datos de entrenamiento después de la imputación:")
print(train.isnull().sum())

print("\nValores faltantes en el conjunto de datos de prueba después de la imputación:")
print(test.isnull().sum())



Valores faltantes en el conjunto de datos de entrenamiento después de la imputación:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

Valores faltantes en el conjunto de datos de prueba después de la imputación:
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


c) Ajustar el primer modelo de regresión logística.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Convertir variables categóricas a numéricas (por ejemplo, 'Sex' y 'Embarked')
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})

train['Embarked'] = train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
test['Embarked'] = test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Separar características y etiqueta en el conjunto de datos de entrenamiento
X = train.drop(['Survived', 'Name', 'Ticket', 'PassengerId'], axis=1)
y = train['Survived']

# Dividir los datos de entrenamiento en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado de características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Ajustar el modelo de regresión logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Predicciones y evaluación del modelo
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy del modelo:", accuracy)
print("Classification Report:\n", classification_report(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))


Accuracy del modelo: 0.7988826815642458
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.85      0.83       105
           1       0.77      0.73      0.75        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179

Confusion Matrix:
 [[89 16]
 [20 54]]


d) Entrenar y determinar el nivel de accuracy del primer modelo.

In [8]:
# Preprocesar el conjunto de datos de prueba
X_test = test.drop(['Name', 'Ticket', 'PassengerId'], axis=1)
X_test = scaler.transform(X_test)

# Predicciones en el conjunto de prueba
test_predictions = model.predict(X_test)

# Crear un DataFrame con los resultados
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_predictions
})

# Guardar el archivo de resultados
submission.to_csv('titanic_predictions.csv', index=False)
